#Installing and Importing Packages

In [1]:
!pip install dwave.system
!pip install dwave.cloud
!pip install dwave.samplers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.6/305.6 kB 26.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement dwave.cloud (from versions: none)
ERROR: No matching distribution found for dwave.cloud


In [2]:
import numpy as np
from collections import defaultdict
from dimod import SimulatedAnnealingSampler
import random
import time
import networkx as nx
import matplotlib.pyplot as plt
from bokeh.palettes import Spectral

# Постановка задачи
Есть n предметов и рюкзак. У каждого есть вес и стоимость. Наша задача - максимизировать стоимость предметов

#Brute Force

In [3]:
# items_values = {"⚽️": 8, "💻": 47, "📸": 10, "📚": 5, "🎸": 16}
# values_list = [8, 47, 10, 5, 16]

# items_weight = {"⚽️": 3, "💻": 11, "📸": 14, "📚": 19, "🎸": 5}
# weights_list = [3, 11, 14, 19, 5]

# maximum_weight = 26

# items_values = {"⚽️": 10, "💻": 10, "📸": 10, "📚": 10, "🎸": 10}
items_values = {i : i for i in range(random.randint(0, 10))}
values_list = [items_values[i] for i in range(len(items_values))]

items_weight = {i : i for i in range(random.randint(0, 10))}
weights_list = [items_values[i] for i in range(len(items_values))]

print(*values_list)
print(*weights_list)

maximum_weight = 50

0 1 2 3 4 5 6
0 1 2 3 4 5 6


In [4]:
def sum_weight(bitstring, items_weight):
    weight = 0
    for n, i in enumerate(items_weight):
        if bitstring[n] == "1":
            weight += i
    return weight


def sum_values(bitstring, items_value):
    value = 0
    for n, i in enumerate(items_value):
        if bitstring[n] == "1":
            value += i
    return value

items = list(items_values.keys())
n_items = len(items)
combinations = {}
max_value = 0
for case_i in range(2**n_items):  # all possible options
    combinations[case_i] = {}
    bitstring = np.binary_repr(
        case_i, n_items
    )  # bitstring representation of a possible combination, e.g, "01100" in our problem means bringing (-💻📸--)
    combinations[case_i]["items"] = [items[n] for n, i in enumerate(bitstring) if i == "1"]
    combinations[case_i]["value"] = sum_values(bitstring, values_list)
    combinations[case_i]["weight"] = sum_values(bitstring, weights_list)
    # save the information of the optimal solution (the one that maximizes the value while respecting the maximum weight)
    if (
        combinations[case_i]["value"] > max_value
        and combinations[case_i]["weight"] <= maximum_weight
    ):
        max_value = combinations[case_i]["value"]
        optimal_solution = {
            "items": combinations[case_i]["items"],
            "value": combinations[case_i]["value"],
            "weight": combinations[case_i]["weight"],
        }


print(
    f"The best combination is {optimal_solution['items']} with a total value: {optimal_solution['value']} and total weight {optimal_solution['weight']} "
)

The best combination is [1, 2, 3, 4, 5, 6] with a total value: 21 and total weight 21 


In [5]:
# def time_to_solution(n, time_single_case):
#     """
#         n (int): number of variables
#         time_single_case (float): time to solve a single case
#     """
#     return time_single_case * 2 ** n

# time_per_case = 1e-9 # time to execute a single case in seconds
# sec_day = 3600 * 24 # seconds in a day
# sec_year = sec_day * 365 # seconds in a year

# print(
#     f"- For 10 items, 2^10 cases, we need {time_to_solution(2, time_per_case)} seconds."
# )

# DP Soluton

In [36]:
def dp_solution(num_of_items, weights, costs, capacity):
    dp = [[0 for x in range(capacity + 1)] for x in range(num_of_items + 1)]

    for i in range(num_of_items + 1):
        for weight in range(capacity + 1):
            if i == 0 or weight == 0:
                dp[i][weight] = 0
            elif weights[i - 1] <= weight:
                dp[i][weight] = max(costs[i-1]
                            + dp[i - 1][weight - weights[i-1]],
                            dp[i - 1][weight])
            else:
                dp[i][weight] = dp[i - 1][weight]

    cur_item = num_of_items
    cur_weight = capacity
    used_items = [0 for i in range(num_of_items)]
    while (cur_item != 0 and cur_weight != 0):
        if (dp[cur_item - 1][cur_weight] == dp[cur_item][cur_weight]):
            cur_item -= 1;
        else:
            cur_weight -= weights[cur_item - 1]
            used_items[cur_item - 1] = 1;
            cur_item -= 1;

    return used_items

In [7]:
# 5
# 3 11 14 19 5
# 8 47 10 5 16
# 26

num_of_items = int(input())
weights = list(map(int, input().split()))
costs = list(map(int, input().split()))
capacity = int(input())
used_items = dp_solution(num_of_items, weights, costs, capacity)

5
3 11 14 19 5
8 47 10 5 16
26
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 
0 0 0 8 8 8 8 8 8 8 8 47 47 47 55 55 55 55 55 55 55 55 55 55 55 55 55 
0 0 0 8 8 8 8 8 8 8 8 47 47 47 55 55 55 55 55 55 55 55 55 55 55 57 57 
0 0 0 8 8 8 8 8 8 8 8 47 47 47 55 55 55 55 55 55 55 55 55 55 55 57 57 
0 0 0 8 8 16 16 16 24 24 24 47 47 47 55 55 63 63 63 71 71 71 71 71 71 71 71 
5 26
57 71
4 21
55 55
3 21
55 55
2 21
8 55
1 10
0 8


In [8]:
def get_result(weights, costs, used_items):
    used_weight = sum([weights[i] * used_items[i] for i in range(len(weights))])
    cost = sum([costs[i] * used_items[i] for i in range(len(weights))])
    return used_weight, cost

In [9]:
print(*get_result(weights, costs, used_items))

19 71


In [10]:
print(*used_items)

1 1 0 0 1


#QUBO Solution

In [11]:
items_values = {"⚽️": 8, "💻": 47, "📸": 10, "📚": 5, "🎸": 16}
values_list = [8, 47, 10, 5, 16]

items_weight = {"⚽️": 3, "💻": 11, "📸": 14, "📚": 19, "🎸": 5}
weights_list = [3, 11, 14, 19, 5]

max_weight = 26

In [12]:
def timer(ret=False):
    def wrapper(func):
        def wrapped(*args):
            start_time = time.perf_counter_ns()
            res = func(*args)
            ms = (time.perf_counter_ns() - start_time) * 1e-6

            if ret:
                return res, ms

            print(ms, "ms")
            return res

        return wrapped

    return wrapper

In [22]:
def make_qubo(costs, weights, max_weight):
    q = defaultdict(int)
    A = 2
    for i in range(len(weights)):
        if i < len(costs):
            q[(i, i)] += A * weights[i] * (weights[i] - 2 * max_weight) - costs[i]
        else:
            q[(i, i)] += A * weights[i] * (weights[i] - 2 * max_weight)
        for j in range(i + 1, len(weights)):
            q[(i, j)] += 2 * A * weights[i] * weights[j]
    return q

def solve_qubo(costs, weights, max_weight):
    q = make_qubo(costs, weights, max_weight)

    sampler = SimulatedAnnealingSampler()
    sampleset = sampler.sample_qubo(q)

    ans = sampleset.first.sample
    ans_array = [ans[i] for i in range(len(costs))]
    ans_weight = sum([ans_array[i] * weights[i] for i in range(len(costs))])
    ans_cost = sum([ans_array[i] * costs[i] for i in range(len(costs))])

    return ans_array

In [14]:
# 5
# 8 47 10 5 16
# 3 11 14 19 5
# 26

num_items = int(input())
costs = list(map(int, input().split()))
weights = list(map(int, input().split()))
max_weight = int(input())

n = round(np.ceil(np.log2(max_weight)))
weights += [2 ** i for i in range(n)]

make_qubo(costs, weights, max_weight)

ans_array, ans_weight, ans_cost = solve_qubo(costs, weights, max_weight);

print('Found vector: ', *ans_array)
print('Total cost: ', ans_cost)
print('Total weight: ', ans_weight)

5
8 47 10 5 16
3 11 14 19 5
26
Found vector:  1 1 0 0 1
Total cost:  71
Total weight:  19


In [89]:
# 5
# 8 47 10 5 16
# 3 11 14 19 5
# 26

num_items = int(input())
costs = list(map(int, input().split()))
weights = list(map(int, input().split()))
capacity = int(input())

n = round(np.ceil(np.log2(capacity)))
weights += [2 ** i for i in range(n)]

solve_qubo(costs, weights, capacity)

5
8 47 10 5 16
3 11 14 19 5
26


[1, 1, 0, 0, 1]

In [93]:
ans_cost, time = timer(True)(solve_qubo)(costs, weights, capacity)

AttributeError: 'list' object has no attribute 'perf_counter_ns'

In [94]:
time, ans_cost = timer(True)(dp_solution)(num_of_items, weights, costs, max_weight)

AttributeError: 'list' object has no attribute 'perf_counter_ns'

In [84]:
print(ans_cost)

1257.023757


In [76]:
def tests():
    for num_of_items in range(2, 100):
        accuracy = 0
        num_of_tests = 10
        qubo_time = 0
        dp_time = 0
        for test in range(num_of_tests):
            costs = [random.randint(1, num_of_items) for i in range(num_of_items)]
            weights = [random.randint(1, num_of_items) for i in range(num_of_items)]
            capacity = num_of_items * random.randint(1, num_of_items / 2)
            dp_ans, time = timer(True)(dp_solution)(num_of_items, weights, costs, capacity)
            dp_time += time
            qubo_ans, time = timer(True)(solve_qubo)(costs, weights, capacity)
            qubo_time += time
            if (all(dp_ans == qubo_ans)):
                accuracy += 1
        accuracy /= num_of_tests
        print(num_of_items, accuracy, "qubo_time = ", qubo_time, "dp_time = ", dp_time)

In [82]:
tests()

AttributeError: 'list' object has no attribute 'perf_counter_ns'